## MPI Messaging and Deadlock

### Point-to-Point Messaging

* This is the fundamental operation in MPI
* Send a message from one process to another
  * Blocking I/O
  * Blocking provides built in synchronization
  * Blocking can lead to deadlock
* Send and receive, let's do an example in `nodeadlock.c`
  * this program passes a "token" around a ring of processes
  * ring defined by the program
  
<img src="https://www.google.com/url?sa=i&source=images&cd=&ved=2ahUKEwjAuI_k8frgAhVN1lkKHehFDFwQjRx6BAgBEAU&url=https%3A%2F%2Fcommons.wikimedia.org%2Fwiki%2FFile%3AMPI_Ring_topology.png&psig=AOvVaw3POQjDOb7xbbSTKYvhW635&ust=1552421193100793" width=400 />
  
* What's in a message?
  * first three arguments specify content
  * then location (receive from or send to)
  * message metadata
  * and a "communicator" which is a virtual network
```c
int MPI_Send ( 
  	void* sendbuf, 
	int count, 
	MPI_Datatype datatype,
	int dest,
	int tag,
	MPI_Comm comm )
    
int MPI_Recv ( 
  	void* recvbuf, 
	int count, 
	MPI_Datatype datatype,
	int source,
	. . . )
```
* All MPI data are arrays
  * Where is it? `void *`
  * How many? `count`
  * What type? `MPI_Datatype`
